In [1]:
import json

In [2]:
import tweepy, keys

In [3]:
import pandas as pd

In [4]:
from tweetutilities import get_API

In [5]:
from locationlistener import LocationListener

In [6]:
from pymongo import MongoClient

In [7]:
from tweetutilities import get_geocodes

In [8]:
api = get_API()

2021-11-14 17:09:52.504 
  command:

    streamlit run C:\Users\liewg\anaconda3\envs\stockbrew_env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\liewg\\Documents\\Self Learning\\Projects\\cryptowatch\\.streamlit\\secrets.toml'

In [9]:
tweets = [] 

In [ ]:
counts = {'total_tweets': 0, 'locations': 0}

In [10]:
atlas_client = MongoClient(keys.mongo_connection_string)

In [11]:
db = atlas_client.crypto

In [ ]:
location_listener = LocationListener(api, counts_dict=counts, 
    tweets_list=tweets, topic='bitcoin', database = db, limit=10)

In [ ]:
stream = tweepy.Stream(auth=api.auth, listener=location_listener)

In [ ]:
stream.filter(track=['bitcoin'], languages=['en'], is_async=False)  

In [ ]:
counts['total_tweets']

In [ ]:
counts['locations']

In [ ]:
bad_locations = get_geocodes(tweets)

In [ ]:
bad_locations

In [ ]:
tweets

In [ ]:
df = pd.DataFrame(tweets)

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
tt_dict = df.to_dict('records')

In [ ]:
tt_dict

In [ ]:
for t in tt_dict:
    json_object = json.dumps(t)
    json_data = json.loads(json_object)  # convert string to JSON
    db.bitcoin.insert_one(json_data)  # store in tweets collection

In [ ]:
db_df = pd.DataFrame(list(db.bitcoin.find()))

In [ ]:
db_df

In [ ]:
db_df.sort_values('created_at')

In [ ]:
location = 'Mars'

In [ ]:
db_location = db_df[db_df['location']==location]

In [ ]:
db_location

In [ ]:
import folium

In [ ]:
usmap = folium.Map(location=[39.8283, -98.5795], 
                    zoom_start=5, detect_retina=True)

In [ ]:
for t in db_df.itertuples():
     # text = ': '.join([t.screen_name, t.text, t.created_at])
     text = f'{t.screen_name}: {t.text}, Created at: {t.created_at}'
     popup = folium.Popup(text, parse_html=True)
     marker = folium.Marker((t.latitude, t.longitude), 
                            popup=popup)
     marker.add_to(usmap)

In [ ]:
usmap